In [ ]:
import requests
import bs4
import pandas as pd
from time import sleep

# Задание 1.

Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в заголовках которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = [‘python’, ‘парсинг’]`

В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`



In [ ]:
KEYWORDS = ['python', 'парсинг']

post_list_url = 'https://habr.com/ru/all/'
base_url = 'https://habr.com'
post_df = pd.DataFrame(columns=['dt', 'title', 'href'])

for bs_article in bs4.BeautifulSoup(features = 'html.parser', markup = requests.get(post_list_url).text).find_all('article', class_ = 'tm-articles-list__item'):
    bs_title_a = bs_article.find('a', class_ = 'tm-article-snippet__title-link')
    try:
        if (any(map(lambda kw: kw in str(bs_title_a.string).lower(), KEYWORDS))):

            title = bs_title_a.string

            href = base_url + bs_title_a['href']

            dt = None
            try:
                bs_dt_txt = bs_article.find('span', class_ = 'tm-article-snippet__datetime-published').find('time')
                dt = pd.to_datetime(bs_dt_txt['datetime'], format = '%Y-%m-%dT%H:%M:%S') 
            except Exception as e: print(e)

            post_df = post_df.append({'dt': dt, 'title': title, 'href': href}, ignore_index = True)

    except Exception as e: print(e)

post_df

,dt,title,href
0,2022-01-09 13:55:09+00:00,"Собираем свой chromium/electron, с блекджеком ...",https://habr.com/ru/post/599679/


## Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`

In [ ]:
KEYWORDS = ['python', 'парсинг']

post_list_url = 'https://habr.com/ru/all/'
base_url = 'https://habr.com'
post_df = pd.DataFrame(columns=['dt', 'title', 'href', 'body'])

for bs_art_prevw in bs4.BeautifulSoup(features = 'html.parser', markup = requests.get(post_list_url).text).find_all('article', class_ = 'tm-articles-list__item'):
    bs_title_a = bs_art_prevw.find('a', class_ = 'tm-article-snippet__title-link')
    try:

            href = base_url + bs_title_a['href']
            
            body = None
            try:
                body_div = bs4.BeautifulSoup(features = 'html.parser', markup = requests.get(href).text).find('div', id = 'post-content-body')
                body = '\n'.join(body_div.strings)
                body = body.lower()
                sleep(0.1)
            except Exception as e: print(e)
            if not body or not any(map(lambda kw: kw in body, KEYWORDS)):
                continue
            
            title = bs_title_a.string


            dt = None
            try:
                bs_dt_txt = bs_art_prevw.find('span', class_ = 'tm-article-snippet__datetime-published').find('time')
                dt = pd.to_datetime(bs_dt_txt['datetime'], format = '%Y-%m-%dT%H:%M:%S') 
            except Exception as e: print(e)

            post_df = post_df.append({'dt': dt, 'title': title, 'href': href, 'body': body}, ignore_index = True)

    except Exception as e: print(e)
    finally: print('Processed an article.')

post_df

Processed an article.
Processed an article.
Processed an article.
Processed an article.
Processed an article.
Processed an article.
Processed an article.
Processed an article.
Processed an article.
Processed an article.
Processed an article.
Processed an article.
Processed an article.
Processed an article.
Processed an article.
Processed an article.
Processed an article.
Processed an article.
Processed an article.
Processed an article.


,dt,title,href,body
0,2022-01-09 10:10:31+00:00,TLS 1.2 (протокол безопасности транспортного у...,https://habr.com/ru/post/599059/,"t. dierks, e. rescorla\n \nпротокол безопаснос..."
1,2022-01-09 06:32:45+00:00,Бомба Log4j и кризис опенсорса,https://habr.com/ru/company/mvideo/blog/599689/,\n\n\r\nучитывая стремительную цифровизацию би...


# Задание 2.

Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Check](https://www.avast.com/hackcheck/). Список email-ов задаем переменной в начале кода:

`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи “скрытого” API. Внимательно изучите post-запросы.



In [ ]:
import json

Лучшее, что мне удалось сделать, так это отправка жестко подобранного запроса сразу после загрузки страницы в браузере с результатами утечки по какому-либо email.

Код не сработает из colab.research.google.com, нужно запускать локально.

In [ ]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']

basic_hdr = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.5',
    'Cache-Control': 'no-cache',
    'Pragma': 'no-cache',
    'DNT': '1',
    'Host': 'identityprotection.avast.com',
    # Отследить этот заголовк в браузере!!!
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:95.0) Gecko/20100101 Firefox/95.0',
    'Origin': 'https://www.avast.com',
    'Referer': 'https://www.avast.com/',
    'Connection': 'keep-alive',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'no-cors',
    'Sec-Fetch-Site': 'same-site',
}

# get the emails' info
avast_qry_hdr = dict(basic_hdr)
avast_qry_hdr.update({
    'Vaar-Version': '0',
    'Vaar-Header-App-Build-Version': '1.0.0',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Content-Type': 'application/json;charset=utf-8',
})
email_info_resp = requests.post('https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data', json = {'emailAddresses': EMAIL}, headers = avast_qry_hdr)

if email_info_resp.headers['Vaar-Status'] != '0':
    print (f'unauthorized-data:\n\tBad status: {email_info_json.headers["Vaar-Status"]}')
else:
    email_info_json = email_info_resp.json()

Воспроизведу здесь ответ от сервера, т.к. из colab его не получить.

In [ ]:
email_info_json = '''{"breaches":{"17110":{"breachId":17110,"site":"azcentral.com","recordsCount":705538,"description":"At an unconfirmed date, online Arizona newspaper \\"AZ Central\\" was allegedly breached. The stolen data contains passwords and email addresses. This breach is being publicly shared on the internet.","publishDate":"2020-01-03T00:00:00Z","statistics":{"usernames":0,"passwords":702971,"emails":705538}},"37177":{"breachId":37177,"site":"forums.vkmonline.com","recordsCount":825654,"description":"At an unconfirmed date, the Russian-language music forum VKM was allegedly breached. The stolen data contains passwords, IPs, email addresses, usernames and additional personal information. This breach is being privately shared on the internet.","publishDate":"2021-02-11T00:00:00Z","statistics":{"usernames":825566,"passwords":825654,"emails":824936}},"41":{"breachId":41,"site":"dropbox.com","recordsCount":68591031,"description":"Cloud storage company Dropbox suffered a major data breach in 2012. Criminals leveraged stolen employee login credentials to access a project document containing user email addresses and hashed passwords. Thanks to employees\' passwords reuse between work and personal accounts, several matches were exploited, which ultimately lead to the full scope of the breach. It wasn\'t until August 2016 that Dropbox confirmed over 68 million user credentials had been breached in 2012. In response, DropBox enforced a password reset for its users. ","publishDate":"2016-10-24T00:00:00Z","statistics":{"usernames":0,"passwords":68591031,"emails":68591031}},"17009":{"breachId":17009,"site":"zynga.com","recordsCount":216159495,"description":"In September 2019, the game developer Zynga was allegedly breached. The stolen data contains usernames, passwords, salts, email addresses and additional personal information. This breach is being privately shared on the internet.","publishDate":"2019-10-17T00:00:00Z","statistics":{"usernames":214648143,"passwords":134762791,"emails":159032663}},"12":{"breachId":12,"site":"vk.com","recordsCount":110121799,"description":"Popular Russian social networking platform VKontakte was breached in late 2012. Over 100 million clear-text passwords were compromised in the breach. Breached credential sets included victims\' e-mail addresses, passwords, dates of birth, phone numbers and location details. The credential set was advertised on a dark web marketplace as of June 2016 for a price of one bitcoin. ","publishDate":"2016-10-29T00:00:00Z","statistics":{"usernames":0,"passwords":107425364,"emails":104247327}},"3":{"breachId":3,"site":"adobe.com","recordsCount":152046506,"description":"In October of 2013, criminals penetrated Adobe\'s corporate network and the stole source code for several of its software products. The affected products included Adobe\'s ColdFusion web application platform as well as the Acrobat suite of products. Adobe asserts that criminals also accessed nearly three-million customer credit card records and stole login data for an undisclosed number of Adobe user accounts.","publishDate":"2016-10-21T00:00:00Z","statistics":{"usernames":0,"passwords":129430596,"emails":152046506}},"13662":{"breachId":13662,"site":"netlog.com","recordsCount":52904632,"description":"Netlog (formerly known as Facebox and Bingbox) was a Belgian social networking website specifically targeted at the global youth demographic. On Netlog, members could create their own web page, meet new people, chat, play games, share videos and post blogs.\\r\\n\\r\\nIn July 2018, Netlog sent out an email informing users registered before December 2012 that there was a security breach of their user database.","publishDate":"2018-02-18T00:00:00Z","statistics":{"usernames":0,"passwords":52902665,"emails":52904632}},"3520":{"breachId":3520,"site":"globalreach.eu","recordsCount":921298,"description":"In 2016, Global Reach Technology\'s database was allegedly breached. The stolen records are being shared among private cirlces on the darknet.","publishDate":"2017-03-15T00:00:00Z","statistics":{"usernames":13492,"passwords":921298,"emails":907696}},"3164":{"breachId":3164,"site":"cfire.mail.ru","recordsCount":12880232,"description":"In July and August of 2016, two criminals carried out attacks on three separate forums hosted by Mail.ru, including CFire. The hackers used known SQL injection vulnerabilities found in older vBulletin forum software to obtain access to the databases. Shortly after the breach itself, the contents of CFire\'s database were leaked publicly. The database contains usernames, email addresses, and MD5 hashed passwords for just under 13 million users.","publishDate":"2017-02-14T00:00:00Z","statistics":{"usernames":12880232,"passwords":12880222,"emails":12845820}},"2961":{"breachId":2961,"site":"cdprojektred.com","recordsCount":1871459,"description":"In March 2016, CDProjektRed.com.com\'s forum database was breached. The attacker may have exploited a vulnerability in vBulletin. The stolen data contains over 1,870,000 user records including associated usernames, email addresses, hashed passwords, and salts. The leaked credentials are being shared and sold privately on the dark web.","publishDate":"2017-01-31T00:00:00Z","statistics":{"usernames":1871410,"passwords":1871457,"emails":1871289}},"15":{"breachId":15,"site":"imesh.com","recordsCount":50913222,"description":"In June 2016, a cache of over 51 million user credentials from the online sharing service iMesh appeared for sale on a dark web marketplace.  The database contained user email addresses, usernames, passwords, IP addresses and location.","publishDate":"2016-10-23T00:00:00Z","statistics":{"usernames":50909015,"passwords":50913222,"emails":50913222}},"3669":{"breachId":3669,"site":"youku.com","recordsCount":94244880,"description":"Youku is a large Chinese video content company whose user database was allegedly breached in 2016. The compromised data later appeared for sale on a dark web marketplace April 2017. The dark web listing advertised the sale of 100M user accounts.","publishDate":"2017-03-24T00:00:00Z","statistics":{"usernames":0,"passwords":94238535,"emails":94244880}},"17670":{"breachId":17670,"site":"wishbone.io","recordsCount":37475476,"description":"In January 2020, the online poll website Wishbone was allegedly breached. The stolen data contains usernames, passwords, email addresses and additional personal information. This breach is being privately shared on the internet.","publishDate":"2020-05-28T00:00:00Z","statistics":{"usernames":31111127,"passwords":10648734,"emails":10234249}},"13094":{"breachId":13094,"site":"myheritage.com","recordsCount":110041653,"description":"In October 2017, a customer database belonging to online genealogy platform MyHeritage was breached. The stolen records included email addresses and SHA-1 hashed passwords, as well as sensitive PII. The data has been sold on at least one dark web market and is also being privately shared amongst several criminal groups.","publishDate":"2017-11-04T00:00:00Z","statistics":{"usernames":0,"passwords":110041647,"emails":110041653}},"16768":{"breachId":16768,"site":"canva.com","recordsCount":137434010,"description":"In May 2019, graphic-design site Canva\'s database was allegedly breached. The stolen data contains usernames, passwords, email addresses and additional personal information. This breach is being privately shared on the internet.","publishDate":"2019-06-13T00:00:00Z","statistics":{"usernames":137304104,"passwords":61348974,"emails":137402835}},"3176":{"breachId":3176,"site":"parapa.mail.ru","recordsCount":5029003,"description":"In July and August 2016, two criminals executed attacks against three separate forums hosted by Mail.ru including the Russian forum Parapa. Shortly after the breach occurred, the contents of Parapa\'s database were leaked publicly. The database contains usernames, email addresses, and hashed passwords for around 5 million users.","publishDate":"2017-02-14T00:00:00Z","statistics":{"usernames":5029000,"passwords":5029003,"emails":4941344}},"2":{"breachId":2,"site":"linkedin.com","recordsCount":158591429,"description":"In 2012, online professional networking platform LinkedIn fell victim to a breach of its members\' passwords. Four years later, 117 million email and password combinations from that breach appeared for sale on a dark web marketplace. \\n\\nThe leaked passwords had only been protected by unsalted SHA-1 cryptographic hashing, which prompted LinkedIn to enforce salted hashing after the breach. Russian national Yevgeniy Nikulin was accused of the breach and was extradited from the Czech Republic to the United States as of March 2018.","publishDate":"2016-10-21T00:00:00Z","statistics":{"usernames":0,"passwords":111975337,"emails":158591429}},"3587":{"breachId":3587,"site":"rayli.com.cn","recordsCount":4844179,"description":"On an unconfirmed date, Chinese gossip site Rayli\'s user database was allegedly breached. The stolen data contains over 4,850,000 records including email addresses, usernames, passwords, and salts. The compromised data is being shared privately among closed and trusted groups.","publishDate":"2017-03-01T00:00:00Z","statistics":{"usernames":4841886,"passwords":4833694,"emails":4803135}}},"data":{"parapa.mail.ru":{"xxx@x.ru":[{"breachId":3176,"usernameBreached":true,"passwordBreached":true}]},"adobe.com":{"xxx@x.ru":[{"breachId":3,"usernameBreached":false,"passwordBreached":true}],"yyy@y.com":[{"breachId":3,"usernameBreached":false,"passwordBreached":true}]},"cdprojektred.com":{"xxx@x.ru":[{"breachId":2961,"usernameBreached":true,"passwordBreached":true}]},"azcentral.com":{"yyy@y.com":[{"breachId":17110,"usernameBreached":false,"passwordBreached":true}]},"myheritage.com":{"yyy@y.com":[{"breachId":13094,"usernameBreached":false,"passwordBreached":true}]},"rayli.com.cn":{"yyy@y.com":[{"breachId":3587,"usernameBreached":true,"passwordBreached":true}]},"canva.com":{"yyy@y.com":[{"breachId":16768,"usernameBreached":true,"passwordBreached":true}]},"globalreach.eu":{"yyy@y.com":[{"breachId":3520,"usernameBreached":false,"passwordBreached":true}]},"forums.vkmonline.com":{"yyy@y.com":[{"breachId":37177,"usernameBreached":true,"passwordBreached":true}]},"netlog.com":{"yyy@y.com":[{"breachId":13662,"usernameBreached":false,"passwordBreached":true}]},"imesh.com":{"xxx@x.ru":[{"breachId":15,"usernameBreached":true,"passwordBreached":true}],"yyy@y.com":[{"breachId":15,"usernameBreached":true,"passwordBreached":true}]},"zynga.com":{"yyy@y.com":[{"breachId":17009,"usernameBreached":true,"passwordBreached":false}]},"linkedin.com":{"yyy@y.com":[{"breachId":2,"usernameBreached":false,"passwordBreached":true}]},"dropbox.com":{"yyy@y.com":[{"breachId":41,"usernameBreached":false,"passwordBreached":true}]},"wishbone.io":{"yyy@y.com":[{"breachId":17670,"usernameBreached":true,"passwordBreached":true}]},"youku.com":{"yyy@y.com":[{"breachId":3669,"usernameBreached":false,"passwordBreached":true}]},"cfire.mail.ru":{"xxx@x.ru":[{"breachId":3164,"usernameBreached":true,"passwordBreached":true}]},"vk.com":{"xxx@x.ru":[{"breachId":12,"usernameBreached":false,"passwordBreached":true}]}},"summary":{"xxx@x.ru":{"breaches":[3,12,15,2961,3164,3176]},"yyy@y.com":{"breaches":[2,3,15,41,3520,3587,3669,13094,13662,16768,17009,17110,17670,37177]}}}'''

In [ ]:
email_struct = json.loads(email_info_json)

In [ ]:
email_df = pd.DataFrame(columns = ['email', 'dt', 'src', 'desc'])
for src, breached_email in email_struct['data'].items():
    for email, breaches_lst in breached_email.items():
        for accident_data in breaches_lst:
            breach_id = accident_data['breachId']
            dt = email_struct['breaches'][str(breach_id)]['publishDate']
            desc = email_struct['breaches'][str(breach_id)]['description']
            new_entry_df = pd.DataFrame(columns = ['email', 'dt', 'src', 'desc'], data=[[email, dt, src, desc]])
            email_df = pd.concat([email_df, new_entry_df])

In [ ]:
email_df

,email,dt,src,desc
0,xxx@x.ru,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
0,xxx@x.ru,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
0,yyy@y.com,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
0,xxx@x.ru,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
0,yyy@y.com,2020-01-03T00:00:00Z,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
0,yyy@y.com,2017-11-04T00:00:00Z,myheritage.com,"In October 2017, a customer database belonging..."
0,yyy@y.com,2017-03-01T00:00:00Z,rayli.com.cn,"On an unconfirmed date, Chinese gossip site Ra..."
0,yyy@y.com,2019-06-13T00:00:00Z,canva.com,"In May 2019, graphic-design site Canva's datab..."
0,yyy@y.com,2017-03-15T00:00:00Z,globalreach.eu,"In 2016, Global Reach Technology's database wa..."
0,yyy@y.com,2021-02-11T00:00:00Z,forums.vkmonline.com,"At an unconfirmed date, the Russian-language m..."


## Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.

Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

```
GROUP = 'netology’
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
```

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

#### Далее будут приведены результаты локального выполнения кода, т.к. access_token пользователя привязывается к IP (а регистрировать приложение и/или глубоко разбираться в API мне лень:)

In [ ]:
token = 'secret'

In [ ]:
# https://dev.vk.com/method/groups.search
search_group_qry_params = {
                           'q': 'netology',
                           'type': 'group',
                           'future': '0',
                           'market': '0',
                           'sort': '0', # 6 - по кол. польз. не работает (
                           'count': '10',
                           'v': '5.131',
                           'access_token': token
                           }
groups = requests.post(r'https://api.vk.com/method/groups.search', params = search_group_qry_params).json()
groups

{'error': {'error_code': 5,
  'error_msg': 'User authorization failed: invalid access_token (4).',
  'request_params': [{'key': 'q', 'value': 'netology'},
   {'key': 'type', 'value': 'group'},
   {'key': 'future', 'value': '0'},
   {'key': 'market', 'value': '0'},
   {'key': 'sort', 'value': '0'},
   {'key': 'count', 'value': '10'},
   {'key': 'v', 'value': '5.131'},
   {'key': 'method', 'value': 'groups.search'},
   {'key': 'oauth', 'value': '1'}]}}

In [ ]:
groups = {'response': {'count': 3, 'items': [{'id': 111913632, 'name': 'Контекстная реклама Yandex Direct  Google Ads', 'screen_name': 'yandexdirect_googleads', 'is_closed': 0, 'type': 'group', 'is_admin': 0, 'is_member': 0, 'is_advertiser': 0, 'photo_50': 'https://sun1-21.userapi.com/s/v1/ig2/4BXCbeY4Z9t4RgfE89iEwHV9iZoyJlTELKrnZ8SHVOcjDvLkoUdiKyXwDAbSot3ZpBQlvi6tj0j1-M2ugPLv1P8_.jpg?size=50x50&quality=96&crop=164,39,415,415&ava=1', 'photo_100': 'https://sun1-21.userapi.com/s/v1/ig2/tsafo-WSGOtA7HVeGc58p3_RErDA21gXtobZDaCnO2aD9VM8KB1PlB2GUNxa48cMEo9DthriBNpc6dvkxETYoFqP.jpg?size=100x100&quality=96&crop=164,39,415,415&ava=1', 'photo_200': 'https://sun1-21.userapi.com/s/v1/ig2/qHGKbonoW8u8z5t9_jfKBw_xotByjebkPlbf3QOWWHTtsF4PMR9StxW8TGtNlPuexjtz8igPqr16KDMscapiw4v7.jpg?size=200x200&quality=96&crop=164,39,415,415&ava=1'}, {'id': 30159897, 'name': 'Нетология. Меняем карьеру через образование', 'screen_name': 'netology', 'is_closed': 0, 'type': 'page', 'is_admin': 0, 'is_member': 0, 'is_advertiser': 0, 'photo_50': 'https://sun1-95.userapi.com/s/v1/ig2/dsizD3yB_HAo2onrnk4ZrAFppduuxulcw-RkjhIy4eWjaxFATaFueDyDCMPB5odwQYNqeh_700IA9itm1whxzlmp.jpg?size=50x50&quality=96&crop=0,0,500,500&ava=1', 'photo_100': 'https://sun1-95.userapi.com/s/v1/ig2/q6sxepgprtgCnrodYqfE65W5xRjUv-JRm3d5YHmFh_h4p40HlZBGldW863X5-MkSgaH-rDOatBvgkzIxDJerdEwj.jpg?size=100x100&quality=96&crop=0,0,500,500&ava=1', 'photo_200': 'https://sun1-95.userapi.com/s/v1/ig2/FYkr1O5WAAmM1C-BND2aQMrIFosV5IUfJFXmTR6wUOHsQZwnIw3qzQ_BSo8B3YmxpRDzWNuNqXRdTzP_mI-6zwfc.jpg?size=200x200&quality=96&crop=0,0,500,500&ava=1'}, {'id': 76868422, 'name': 'Onion.23', 'screen_name': 'onion23', 'is_closed': 0, 'type': 'group', 'is_admin': 0, 'is_member': 0, 'is_advertiser': 0, 'photo_50': 'https://sun1-85.userapi.com/s/v1/if2/8rI2uiVvaK_hDqsxkZ_4Y0CycH9BM9iQmavxnfmN8muNiP7OYxTqhlOd_J_WPbtgsqFEV6s9Wgc-FI3_I_MOTqN9.jpg?size=50x50&quality=96&crop=20,54,360,360&ava=1', 'photo_100': 'https://sun1-85.userapi.com/s/v1/if2/lJXctLMsuD8j8h1Lz9GfvWNRV3KYiGboHhEGWityCrQZQZxICLU9Lv5so4XCXhDBrNZvDN8W0mQItACltzK2sVCE.jpg?size=100x100&quality=96&crop=20,54,360,360&ava=1', 'photo_200': 'https://sun1-85.userapi.com/s/v1/if2/xuMZD_VerJ9PDow5Y-b-BlIujRUktreg6HLj2hmNGkNkKVGOoro6GKAVboe0AbORYHuSbsOlrXekotQfdQDm70Gz.jpg?size=200x200&quality=96&crop=20,54,360,360&ava=1'}]}}
groups

{'response': {'count': 3,
  'items': [{'id': 111913632,
    'is_admin': 0,
    'is_advertiser': 0,
    'is_closed': 0,
    'is_member': 0,
    'name': 'Контекстная реклама Yandex Direct  Google Ads',
    'photo_100': 'https://sun1-21.userapi.com/s/v1/ig2/tsafo-WSGOtA7HVeGc58p3_RErDA21gXtobZDaCnO2aD9VM8KB1PlB2GUNxa48cMEo9DthriBNpc6dvkxETYoFqP.jpg?size=100x100&quality=96&crop=164,39,415,415&ava=1',
    'photo_200': 'https://sun1-21.userapi.com/s/v1/ig2/qHGKbonoW8u8z5t9_jfKBw_xotByjebkPlbf3QOWWHTtsF4PMR9StxW8TGtNlPuexjtz8igPqr16KDMscapiw4v7.jpg?size=200x200&quality=96&crop=164,39,415,415&ava=1',
    'photo_50': 'https://sun1-21.userapi.com/s/v1/ig2/4BXCbeY4Z9t4RgfE89iEwHV9iZoyJlTELKrnZ8SHVOcjDvLkoUdiKyXwDAbSot3ZpBQlvi6tj0j1-M2ugPLv1P8_.jpg?size=50x50&quality=96&crop=164,39,415,415&ava=1',
    'screen_name': 'yandexdirect_googleads',
    'type': 'group'},
   {'id': 30159897,
    'is_admin': 0,
    'is_advertiser': 0,
    'is_closed': 0,
    'is_member': 0,
    'name': 'Нетология. Меняем ка

In [ ]:
# https://dev.vk.com/method/wall.get
wal_get_qry_params = {
    'owner_id': '-30159897',
    # 'domain': '',
    'offset': '0',
    'count': '50',
    'filter': 'owner',
    'extented': '0',
    # 'fields': '',
    'access_token': token,
    'v': '5.131',
}
posts = requests.post(r'https://api.vk.com/method/wall.get', params = wal_get_qry_params).json()
posts_proj = [{'ts': pd.to_datetime(item['date'], unit = 's'), 'text': item['text']} for item in posts['response']['items'] if item['post_type'] == 'post']

In [ ]:
from pandas import Timestamp

In [ ]:
posts_proj = [{'ts': Timestamp('2022-01-12 13:30:00'), 'text': 'Длинные выходные позади, пора восстанавливать рабочий режим 💼\n \nКакие напоминания поставите себе на телефон?'}, {'ts': Timestamp('2022-01-15 09:11:00'), 'text': 'Зачитались статьями из подборки и ушли играть в перевыпуск GTA 🚘\n\n🔹 Зачем NPC разум: искусственный интеллект в играх → https://netolo.gy/hwY\n\n🔹 Эволюция мобильных игр: немного ностальгии и восхищения → https://netolo.gy/hwZ\n\n🔹 Как создаются интересные открытые миры: ключевые принципы → https://netolo.gy/hwV\n\n🔹 «Нужно больше золота»: как зарабатывать в игровой индустрии → https://netolo.gy/hwW \n\n🔹 Почему нельзя найти универсальный сеттинг, который понравится всем → https://netolo.gy/hwX'}, {'ts': Timestamp('2022-01-14 14:00:00'), 'text': 'Поздравляем, вы добрались до конца первой рабочей недели в этом году! Сегодня мы не будем мучить сложными вопросами и просто спросим, как вы? Получилось ли войти обратно в режим? Что с ресурсом? \n\nПишите в комментариях 👇'}, {'ts': Timestamp('2022-01-14 07:27:00'), 'text': 'По данным сервиса по трудоустройству Trud.com, с марта 2020 года вырос спрос на всех специалистов диджитал-маркетинга. Делимся подборкой бесплатных занятий, чтобы помочь вам стать профессионалами: \n\n▪ 17 января, 19:00 — Google Analytics: начало работы\nРазберётесь, что такое веб-аналитика, какие инструменты используют специалисты и как настраивать счётчики в Google Analytics → https://netolo.gy/hxj \n\n▪ 25 января, 17:00 — Интернет-маркетинг: с чего начать\nУзнаете о профессиях в интернет-маркетинге, необходимых навыках и старте карьеры → https://netolo.gy/hxl \n\n▪ 26 января, 19:00 — SMM-менеджер: что делает и куда расти\nСпикер — операционный менеджер одной из популярных соцсетей Антон Астафуров → https://netolo.gy/hxk \n\n▪ 31 января, 19:00 — Тренды: как оформить и вести аккаунт, чтобы он стал успешным\nЭксперт расскажет, что нужно учитывать при оформлении профиля, как заинтересовать аудиторию и где черпать вдохновение → https://netolo.gy/hxm'}, {'ts': Timestamp('2022-01-13 15:31:00'), 'text': 'Дизайнер — одна из самых популярных диджитал-профессий. Но из-за большого количества разных специализаций новички часто не понимают, где они хотят работать и какие навыки для этого нужны. \n\nРассказываем, какие дизайнерские профессии существуют и чем занимаются эти специалисты 👇'}, {'ts': Timestamp('2022-01-13 07:39:00'), 'text': 'Добиться новых вершин в карьере в 2022 году поможет наша подборка видео 🙃'}, {'ts': Timestamp('2022-01-12 07:16:00'), 'text': '18 января стартует бесплатный курс «Как стать программистом»\n\nКаждый день на рынке появляется очень много вакансий для джуниор-специалистов, предложения для программистов — в их числе. За три онлайн-занятия узнаете:\n\n▪ Действительно ли разработчику нужно быть гуру математики\n▪ Какие востребованные сферы разработки существуют\n▪ Как найти стажировку и не облажаться\n\nЗаписаться можно здесь → https://netolo.gy/huA'}, {'ts': Timestamp('2022-01-11 16:01:00'), 'text': 'Охваты, репосты, вовлечённость подписчиков — не пустой звук для тех, кто продвигает личный бренд или ведёт аккаунт компании в соцсетях. \n\nРазбираемся, что такое хороший пост и как писать тексты для соцсетей 👇'}, {'ts': Timestamp('2022-01-11 08:14:00'), 'text': 'Эффективны ли сервисы-опросники?\n□ Да \n□ Нет\n□ Узнаю в статье\n\nСсылка → https://netolo.gy/huT'}, {'ts': Timestamp('2022-01-10 15:33:00'), 'text': 'Извините, не можем написать подводку: копирайтер уже пятый час играет в Doodle Jump → https://netolo.gy/huS'}, {'ts': Timestamp('2022-01-10 08:12:00'), 'text': '13 января стартует бесплатный курс «Системный аналитик: первые шаги к профессии»\n\nСистемный аналитик — востребованная IT-профессия, которая не требует навыков программирования. За 12 дней разберётесь: \n\n🔹 Как объяснить родителям, чем занимается системный аналитик\n🔹 Что такое мэппинг данных, фронтенд и бэкенд и другие непонятные слова \n🔹 Как составить ТЗ разработчику, чтобы обойтись без правок\n\nРегистрируйтесь здесь → https://netolo.gy/huB'}, {'ts': Timestamp('2022-01-09 13:08:00'), 'text': 'Чем заняться зимними вечерами, когда наконец доели салаты, накатались на горках, успели встретиться со всеми близкими? Можно провести время за просмотром классных фильмов. \n\nСобрали подборку мини-сериалов для тех, кто устал пересматривать советские комедии и сагу о Гарри Поттере 👇'}, {'ts': Timestamp('2022-01-06 11:32:00'), 'text': 'Для многих Новый год — время начать новую жизнь. Каждый хоть раз давал себе обещание начать бегать по утрам, читать больше книг или пойти учиться чему-то. \n\nРассказываем, как правильно ставить себе цели, которые точно будут достигнуты 👇'}, {'ts': Timestamp('2022-01-05 11:42:00'), 'text': 'Пересмотрели Гарри Поттера, доели салаты, навестили всех родных и даже сходили на каток — что дальше? Есть парочка идей → https://netolo.gy/huR'}, {'ts': Timestamp('2022-01-04 12:19:00'), 'text': 'В первый рабочий день после долгих праздников обычно тяжело включиться в работу. Мозг отвык решать сложные задачи, а тело не хочет вставать с дивана и ехать в офис.\n\nМы спросили у коллег, как они проводят новогодние каникулы и что помогает им снова войти в рабочий ритм 👇'}, {'ts': Timestamp('2021-12-31 14:00:00'), 'text': '2021 был непростым, но мы все справились ✨\n \nПора уже закончить все дела, отключить гаджеты и провести время с близкими. Обнимашки, беседы, признания в любви — вот это всё ❤\n \nУже два года мы убеждаемся, что способны на всё: общаться друг с другом на расстоянии, учиться, противостоять неопределённости и продолжать усердно работать! Спасибо, что читаете наши посты, смотрите вебинары, делаете домашки — и пишете дипломы наконец! Вы очень большие молодцы 🔥\n \nС наступающим!'}, {'ts': Timestamp('2021-12-31 07:45:00'), 'text': 'Каждый год мы смотрим, как Иван Васильевич меняет профессию, Евгений Трошкин меняет амплуа и становится Доцентом, а Женя Лукашин меняет город и находит счастье 🙃\n\nВ этот раз, помимо любимых комедий, предлагаем посмотреть бесплатные курсы Нетологии — и найти себе новую профессию. Все курсы в записи, поэтому начать просмотр можно в любое время — доступ к ним придёт на вашу почту сразу после регистрации. \n\nПереходите по ссылке и выбирайте курс по душе → https://netolo.gy/ht6'}, {'ts': Timestamp('2021-12-30 13:30:00'), 'text': 'Цифровой гороскоп* на 2022 от наших цифрологов, часть 2 ✨ В год Тигра вас ждёт...\n\n*Цифровой гороскоп является юмористической заметкой и не призывает ни к каким действиям.'}, {'ts': Timestamp('2021-12-30 08:12:17'), 'text': 'Какие продукты лучше исключить из Новогоднего меню, как не переедать на праздниках и вредно ли выпивать пепел с желанием 🍸\nОтветы здесь → https://netolo.gy/huU'}, {'ts': Timestamp('2021-12-29 13:30:00'), 'text': 'Пообщались с цифрологами и собрали цифровой гороскоп* на 2022 год. Вот, что ждёт вас в году Тигра 👇\n \nВторая часть — уже завтра ✨\n\n *Цифровой гороскоп является юмористической заметкой и не призывает ни к каким действиям.'}, {'ts': Timestamp('2021-12-29 07:53:00'), 'text': '19 января запускаем бесплатный интенсив «Тестировщик: начните карьеру в IT» 🔥\n\nУвеличивать качество продукта — от мобильных приложений до сайтов — с каждым днём всё важнее для бизнеса. И важную роль в этом играют именно специалисты по обеспечению качества. За два дня разберётесь:\n\n▪ Какие есть перспективы в профессии\n▪ Как быстро можно найти работу \n▪ Сколько зарабатывает специалист без опыта\n\nСсылку на запись, как всегда, оставляем здесь → https://netolo.gy/hrY'}, {'ts': Timestamp('2021-12-28 15:16:00'), 'text': 'Часто мы загораемся какой-то идеей: получить новую профессию, выучить английский, начать бегать. Сначала всё идёт отлично, но со временем мотивация начинает падать. Из-за этого некоторые цели откладываются, а потом вообще забываются. \n\nРазбираемся, что делать, если почувствовали спад мотивации и нежелание заканчивать важные дела 👇'}, {'ts': Timestamp('2021-12-28 07:19:00'), 'text': 'Заказ принят: рассказываем, кто такой управляющий рестораном → https://netolo.gy/hr9'}, {'ts': Timestamp('2021-12-27 15:41:00'), 'text': 'Хэллоу, ду ю спик инглиш → https://netolo.gy/hrW'}, {'ts': Timestamp('2021-12-27 07:47:00'), 'text': 'Ждём вас на бесплатном курсе «Основы Adobe Photoshop»\n\nКурс специально для тех, кто никогда не открывал Photoshop или умеет совсем немного. За шесть практических занятий научитесь:\n\n▪ Редактировать изображения. Теперь не придётся просить друзей обрабатывать ваши фотографии\n▪ Создавать gif — волшебные фотографии из вселенной Гарри Поттера, только в реальности\n▪ Верстать макеты. Станете экспертом в модульных сетках и направляющих\n\nНачать учиться можно прямо сейчас → https://netolo.gy/hrU'}, {'ts': Timestamp('2021-12-26 08:18:00'), 'text': 'До Нового года осталось всего 5 дней — а значит, можно успеть посмотреть как минимум 5 фильмов для создания праздничного настроения. \n\nСделали подборку сюжетов о том, как в Новый год можно изменить к лучшему свою жизнь и мир вокруг 👇'}, {'ts': Timestamp('2021-12-25 14:00:01'), 'text': 'Рассказываем, чем занималось направление «Высшее образование» в 2021 году 👇'}, {'ts': Timestamp('2021-12-25 08:13:00'), 'text': 'Чтобы в 2022 не выгореть от потока задач, собрали вам подборку статей по работе с проектами:\n\n▪ Лучшие функции 15 популярных систем управления проектами → https://netolo.gy/hrE\n\n▪ Этапы оценки проекта: понятия, методы и полезные инструменты → https://netolo.gy/hrF \n\n▪ Почему срываются проекты и как этого избежать: базовые принципы проектного менеджмента → https://netolo.gy/hrG \n\n▪ Как управлять проектами в Trello на двух досках вместо десяти → https://netolo.gy/hrH'}, {'ts': Timestamp('2021-12-24 14:00:00'), 'text': 'На новогодних каникулах мы не будем устраивать открытых занятий, чтобы вы могли отдохнуть, но большинство бесплатных курсов открыты в любое время 😌\n\nЧем будете заниматься на новогодних каникулах?'}, {'ts': Timestamp('2021-12-24 10:00:01'), 'text': 'Количество профессий в диджитале постоянно растёт, а у бизнеса появляются новые условия и требования к специалистам 🤔\n\nМы опросили наших студентов, которые решились в 2021 году сменить профессию, и соотнесли их опыт с данными по рынку — ростом диджитал-сфер и спросом на специалистов. В результате получилось исследование, которое поможет вам выбрать перспективное направление для профессионального развития.\n\nСкачивайте и узнавайте новое 👇'}, {'ts': Timestamp('2021-12-24 07:29:00'), 'text': 'Сохраняйте подборку бесплатных занятий в январе — поможем начать год продуктивно:\n\n🔹 18 января, 17:00 — Системный администратор: быстрый старт в IT\nСпикер расскажет про IT-профессию, для которой не требуется знание программирования, её востребованности и перспективах → https://netolo.gy/hrO \n\n🔹 18 января, 19:00 — Арт-менеджмент: как управлять проектами в сфере искусства\nЭксперты объяснят, как устроены процессы в арт‑индустрии, чем занимается арт‑менеджер и как им стать → https://netolo.gy/hrQ \n\n🔹 19 января, 17:00 — Как стать специалистом по информационной безопасности с нуля\nПоговорим об индустрии информационной безопасности, технологиях и самой профессии → https://netolo.gy/hrP \n\n🔹 25 января, 17:00 — Интернет-маркетинг: с чего начать\nОбо всём расскажет и проведёт вебинар генеральный директор маркетингового агентства DVIGA Тимофей Белоглазов → https://netolo.gy/hrR \n\n🔹 27 января, 20:00 — Дизайнер интерфейсов: с чего начать погружение в UX/UI-дизайн\nПоговорим об интуитивном интерфейсе, требованиях к профессии и принципах работы UX- и UI-дизайнеров → https://netolo.gy/hrS'}, {'ts': Timestamp('2021-12-23 15:46:00'), 'text': 'Каждый день мы соприкасаемся с разными цифровыми продуктами: это сайты, приложения, видеоролики и даже аккаунты в соцсетях. Нравятся они нам или нет — во многом зависит от визуальной составляющей. \n\nРазбираемся, что делает дизайн актуальным и какие тренды помогут привлечь внимание пользователей 👇'}, {'ts': Timestamp('2021-12-23 08:24:00'), 'text': 'Чтобы защитить персональные данные в сети, не нужно ездить на станцию «Люблино» — просто прочитайте статью → https://netolo.gy/hrN'}, {'ts': Timestamp('2021-12-22 13:30:00'), 'text': 'Финишная прямая до новогодних каникул — решаем последнюю задачу в этом году 🤍'}, {'ts': Timestamp('2021-12-22 08:12:00'), 'text': 'Залетаем в реки с нашим бесплатным курсом по продвижению коротких роликов 😉\n\nФормат коротких видео становится популярным в каждой соцсети: бизнес продвигает свои продукты, а блогеры — личный бренд. На курсе научитесь:\n\n▫ Оформлять аккаунт и привлекать больше подписчиков\n▫ Учитывать особенности алгоритмов, чтобы попадать в рекомендации\n▫ Делать простой монтаж\n\nЗаписаться можно здесь → https://netolo.gy/hrJ'}, {'ts': Timestamp('2021-12-21 15:33:00'), 'text': 'Только задумайтесь, как много в ваших любимых сериалах героев с непростым характером. Многие из них вполне достойны премии «За заслуги против счастливого конца и морального благополучия любимых персонажей».\n\nВместе с нашими друзьями из [club40234355|студии коммуникаций BEsmart] составили шортлист харизматичных манипуляторов, которые специализируются на кознях и токсичной коммуникации 👇\n\nДополняйте список в комментариях и сохраняйте пост — пригодится, чтобы было легче обходить стороной ядовитых личностей в реальной жизни.'}, {'ts': Timestamp('2021-12-21 07:47:00'), 'text': 'В очередных видео наш любимый Кирилл рассказывает про новые технологии 👇'}, {'ts': Timestamp('2021-12-20 15:46:00'), 'text': 'Маркетологи работают, чтобы потребитель закрыл свою потребность в нужное время и в нужном месте. Эта статья чуть-чуть закроет вашу потребность в маркетинговых знаниях → https://netolo.gy/hrt'}, {'ts': Timestamp('2021-12-20 07:08:00'), 'text': 'Учитесь на бесплатном курсе «Data Science: будущее для каждого» в удобное время 📊\n\nМы в Нетологии верим, что в Data Science может работать каждый. Направлений в этой области много, а значит — много возможностей проявить свои таланты. За три дня разберётесь в направлениях сферы работы с данными, узнаете обязательные навыки и инструментарий, поймёте, как начать свою карьеру 🎯\n\nНачать можно даже сейчас → https://netolo.gy/hri'}, {'ts': Timestamp('2021-12-19 08:32:00'), 'text': 'У термина work-life balance нет точного перевода на русский язык. Так называют баланс между работой и другими сферами жизни. \n\nСделали подборку лекций TED о том, как научиться отдыхать от рабочих задач и находить время для личных дел 👇'}, {'ts': Timestamp('2021-12-18 14:00:00'), 'text': 'Рассказываем, где мы публиковались в последнее время ❄\n\n🔹 Спецпроекты: \n\n— Нетология вместе с Мастерской выпустили вторую часть спецпроекта, посвящённому профессии Data-журналист — о трендах и перспективах дата-журналистики в России → https://netolo.gy/hrZ \n\n🔹 Публикации:\n\n— Анна Чащина, директор по маркетингу и продюсер онлайн-курсов инкубатора специалистов в образовании EdMarket, подготовила колонку и рассказала о том, кто такие продюсеры онлайн-курсов, какие знания и навыки входят в их компетенции и как освоить эту специализацию и начать карьеру в EdTech → https://netolo.gy/hr1 \n\n— Команды учебно-методического отдела и клиентской службы Нетологии рассказали о значимости живого общения студента с преподавателями и специалистами сопровождения в процессе онлайн-обучения, а также о том, как оно реализуется на курсах в Нетологии → https://netolo.gy/hr2 \n\n— Юлия Княжанская, руководитель направления MBA в Нетологии, рассказала о том, как инвестировать в самообразование в формате бизнес-проекта → https://netolo.gy/hr3\n\n— Выпускники Нетологии, которые столкнулись с карьерными трудностями из-за пандемии, поделились своими историями успеха переобучения на онлайн-курсах и первых шагах в новой профессии → https://netolo.gy/hr4 \n\n— Выпускник Станислав Чирков также рассказал о том, как решился на карьерные перемены и переквалифицировался из военного в коммерческого редактора → https://netolo.gy/hr5 \n\n🔹 Подкасты: \n\n— В ноябре вышел шестой выпуск в подкасте «Инфа 100%»! Напомним, что в течение нескольких выпусков ведущие живо и смешно рассуждают об онлайн-обучении и программах на курсах, делятся впечатлениями о собственном опыте обучения в Нетологии → https://netolo.gy/hr6 \n\n— Владимир Ковтун, преподаватель Нетологии направления «Дизайн и UX» и также продюсер и гейм-дизайнер компании Ciliz, вместе с ведущими подкаста «Не занесли» разбирают самую популярную копипасту рунета 2000-х — письмо геймдизайнера-мечтателя, написанное в компанию по разработке игр. А заодно и обсуждают, как правильно написать такое письмо → https://netolo.gy/hr7 \n\n— Светлана Ширшова, выпускница курсов «SMM-менеджер» и «Интернет-маркетолог», стала гостем подкаста «Это непросто» и поделилась своей историей карьерной трансформации. Она рассказала о том, как после 12 лет работы архивариусом в канцелярии она увлекалась диджиталом, освоила в Нетологии две новые профессии и нашла работу по новой специальности → https://netolo.gy/hr8'}, {'ts': Timestamp('2021-12-18 09:03:00'), 'text': 'Сколько зарабатывают разработчики? Зачем работать с блогерами? Как восстановить репутацию бренда после скандала?\n\nСобрали подборку статей, в которых диджитал-специалисты разных профессий отвечают на популярные вопросы:\n\n▪ 10 вопросов сценаристу → https://netolo.gy/hro \n\n▪ 10 вопросов веб-разработчику → https://netolo.gy/hrp \n\n▪ 10 вопросов инфлюенс-маркетологу → https://netolo.gy/hrq\n\n▪ 10 вопросов геймдизайнеру → https://netolo.gy/hrr \n\n▪ 10 вопросов PR-специалисту → https://netolo.gy/hrs'}, {'ts': Timestamp('2021-12-17 14:00:00'), 'text': 'Согласно исследованию SuperJob, в топе ожиданий от работы в 2022 году у россиян повышение зарплаты, стабильность и адекватное руководство 💼\n\nА что от 2022 ждёте вы?'}, {'ts': Timestamp('2021-12-17 08:09:00'), 'text': 'Давно такого не было и вот опять: снова собрали подборку бесплатных курсов 🙃\n\nУчиться можно в любое удобное время:\n\n🔹 Курс «Основы Figma»\nСоздадите рекламный баннер, интерфейс приложения и макет лендинга → https://netolo.gy/hrk \n\n🔹 Курс «Основы Adobe Photoshop»\nНаучитесь верстать макеты, создавать gif и редактировать изображения → https://netolo.gy/hrl \n\n🔹 Курс «\u200b\u200bОсновы Adobe After Effects»\nРазберётесь в слоях, скриптах, масках, ключах и анимации → https://netolo.gy/hrm \n\n🔹 Курс «Основы Adobe Illustrator»\nВо время курса создадите целых 11 работ: от паттернов до логотипов → https://netolo.gy/hrn'}, {'ts': Timestamp('2021-12-16 15:03:00'), 'text': 'Выпускнику Нетологии Станиславу Чиркову удаётся совмещать хобби с работой: он участвует в походах и пишет тексты для производителя туристического снаряжения. Правда, перед этим он прошёл 20 лет военной службы, выгорание и серьёзные проблемы со здоровьем. \n\nРассказываем, как Станислав смог избавиться от вредных привычек и решил начать новую жизнь 👇'}, {'ts': Timestamp('2021-12-16 08:31:00'), 'text': 'Между сеансами на «Человека-паука» и «Матрицу» предлагаем посмотреть наше видео про криптовалюты → https://netolo.gy/hrg'}, {'ts': Timestamp('2021-12-15 13:30:00'), 'text': 'Где искать новогоднее настроение, что подарить близким, как вешать гирлянду — собрали популярные запросы перед Новым годом 🎄\n\nА что ищете в поисковиках перед праздниками вы?'}, {'ts': Timestamp('2021-12-15 07:37:00'), 'text': 'У нас есть бесплатный курс «Диджитал-старт: первый шаг к востребованной профессии» 🌐\n\nКогда вы заказываете продукты на дом или устраиваете онлайн-свидание — вы пользуетесь диджиталом.\n\nНа бесплатном курсе по старту в диджитале вы:\n🔹 Узнаете, кто такие маркетологи, дизайнеры и разработчики\n🔹 Поймёте, в какой сфере развиваться\n🔹 Построите план по развитию карьеры\n\nУчиться можно в любое удобное время → https://netolo.gy/hqO'}, {'ts': Timestamp('2021-12-14 15:57:00'), 'text': 'Всё чаще рабочие встречи проходят в онлайн-формате. Это удобно: можно встретиться с коллегами или заказчиками, не выходя из дома. Но, как и к любой встрече, к таким созвонам лучше готовиться заранее. Рассказываем, что необходимо сделать перед началом видеоконференции 👇\n\nЧитайте о возможностях, форматах и технических особенностях видеоконференций в нашем Медиа → https://netolo.gy/hqA'}, {'ts': Timestamp('2021-12-14 07:13:00'), 'text': 'Зачем SEO-специалисты делают прогноз поискового трафика? Чтобы сформировать у заказчика реалистичные ожидания и не прогадать 🙃\n\nО методах составления прогноза рассказываем в Медиа → https://netolo.gy/hqQ'}]

In [ ]:
posts_proj[0:3]

[{'text': 'Длинные выходные позади, пора восстанавливать рабочий режим 💼\n \nКакие напоминания поставите себе на телефон?',
  'ts': Timestamp('2022-01-12 13:30:00')},
 {'text': 'Зачитались статьями из подборки и ушли играть в перевыпуск GTA 🚘\n\n🔹 Зачем NPC разум: искусственный интеллект в играх → https://netolo.gy/hwY\n\n🔹 Эволюция мобильных игр: немного ностальгии и восхищения → https://netolo.gy/hwZ\n\n🔹 Как создаются интересные открытые миры: ключевые принципы → https://netolo.gy/hwV\n\n🔹 «Нужно больше золота»: как зарабатывать в игровой индустрии → https://netolo.gy/hwW \n\n🔹 Почему нельзя найти универсальный сеттинг, который понравится всем → https://netolo.gy/hwX',
  'ts': Timestamp('2022-01-15 09:11:00')},
 {'text': 'Поздравляем, вы добрались до конца первой рабочей недели в этом году! Сегодня мы не будем мучить сложными вопросами и просто спросим, как вы? Получилось ли войти обратно в режим? Что с ресурсом? \n\nПишите в комментариях 👇',
  'ts': Timestamp('2022-01-14 14:00:00')

In [ ]:
len(posts_proj)

50

In [ ]:
posts_df = pd.DataFrame(posts_proj)
posts_df.sort_values(['ts'], ascending = False)

,ts,text
1,2022-01-15 09:11:00,Зачитались статьями из подборки и ушли играть ...
2,2022-01-14 14:00:00,"Поздравляем, вы добрались до конца первой рабо..."
3,2022-01-14 07:27:00,"По данным сервиса по трудоустройству Trud.com,..."
4,2022-01-13 15:31:00,Дизайнер — одна из самых популярных диджитал-п...
5,2022-01-13 07:39:00,Добиться новых вершин в карьере в 2022 году по...
0,2022-01-12 13:30:00,"Длинные выходные позади, пора восстанавливать ..."
6,2022-01-12 07:16:00,18 января стартует бесплатный курс «Как стать ...
7,2022-01-11 16:01:00,"Охваты, репосты, вовлечённость подписчиков — н..."
8,2022-01-11 08:14:00,Эффективны ли сервисы-опросники?\n□ Да \n□ Нет...
9,2022-01-10 15:33:00,"Извините, не можем написать подводку: копирайт..."
